In [1]:
import argparse
import glob
import os
import time

import numpy as np
import pandas as pd
import cv2

In [2]:
parser = {
    'val_path': '../data/val768/',
    'val_masks_path': '../data/val768_masks/',
    'test_path': '../data/test_hq/',
    'intermediate_path': '../intermediate/output/',
    'output_paths': ['output_unet_41/'],
    'submission_name': 'subu43.csv',
    'log_every': 1000,
}
args = argparse.Namespace(**parser)

args.output_paths = [os.path.join(args.intermediate_path, x)
                     for x in args.output_paths]
args.img_names = sorted([x.split('/')[-1]
                         for x in glob.glob(args.test_path + '/*.jpg')])

In [3]:
def rle_encode(mask_image):
    pixels = mask_image.flatten()
    # We avoid issues with '1' at the start or end (at the corners of 
    # the original image) by setting those pixels to '0' explicitly.
    # We do not expect these to be non-zero for an accurate mask, 
    # so this should not harm the score.
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs

In [4]:
def get_mask(output_paths, index, weights=None):
    if weights is None:
        weights = [1] * len(output_paths)
    
    img = 0
    for i, path in enumerate(output_paths):
        file = os.path.join(path, str(index)+'.jpg')
        image = cv2.imread(file, cv2.IMREAD_GRAYSCALE).astype(np.uint16)
        img = img + image * weights[i]
    img = img / sum(weights)
    return img > 127.5

In [5]:
list_rle = []
init_time = time.time()
for i in range(1, len(args.img_names)+1):
    mask = get_mask(args.output_paths, i)
    rle = rle_encode(mask)
    list_rle.append(' '.join(str(x) for x in rle))
    
    if i % args.log_every == 0:
        print("Done {:6} images in {:.0f}s".format(i, time.time()-init_time))

Done   1000 images in 21s
Done   2000 images in 41s
Done   3000 images in 62s
Done   4000 images in 83s
Done   5000 images in 104s
Done   6000 images in 125s
Done   7000 images in 147s
Done   8000 images in 168s
Done   9000 images in 190s
Done  10000 images in 211s
Done  11000 images in 232s
Done  12000 images in 254s
Done  13000 images in 275s
Done  14000 images in 297s
Done  15000 images in 318s
Done  16000 images in 340s
Done  17000 images in 361s
Done  18000 images in 383s
Done  19000 images in 404s
Done  20000 images in 425s
Done  21000 images in 446s
Done  22000 images in 467s
Done  23000 images in 488s
Done  24000 images in 510s
Done  25000 images in 531s
Done  26000 images in 552s
Done  27000 images in 573s
Done  28000 images in 594s
Done  29000 images in 616s
Done  30000 images in 637s
Done  31000 images in 658s
Done  32000 images in 679s
Done  33000 images in 701s
Done  34000 images in 722s
Done  35000 images in 742s
Done  36000 images in 763s
Done  37000 images in 785s
Done 

In [6]:
df = pd.DataFrame({"img": args.img_names, "rle_mask": list_rle})
submiss_path = os.path.join(args.intermediate_path, args.submission_name)
df.to_csv(submiss_path, index=False)